# Viabill – Analysis Report

This notebook loads precomputed CSVs (SQL outputs) and creates charts for:
1) Portfolio Overview
2) Payment Analysis
3) Vintage Analysis

Data source: `outputs/*.csv`

In [ ]:

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

DATA = Path("outputs")
PLOTS = DATA / "plots"
PLOTS.mkdir(parents=True, exist_ok=True)

def line(df, x, y, title, fname):
    plt.figure()
    plt.plot(df[x], df[y])
    plt.title(title); plt.xlabel(x); plt.ylabel(y)
    if pd.api.types.is_datetime64_any_dtype(df[x]):
        plt.xticks(rotation=45, ha="right")
    plt.tight_layout(); plt.savefig(PLOTS / fname); plt.show()

def bar(df, x, y, title, fname):
    plt.figure()
    plt.bar(df[x], df[y])
    plt.title(title); plt.xlabel(x); plt.ylabel(y)
    if pd.api.types.is_datetime64_any_dtype(df[x]):
        plt.xticks(rotation=45, ha="right")
    plt.tight_layout(); plt.savefig(PLOTS / fname); plt.show()

def parse_month(df, col="month"):
    try:
        df[col] = pd.to_datetime(df[col], format="%Y-%m")
    except Exception:
        try: df[col] = pd.to_datetime(df[col])
        except Exception: pass
    return df


## 1) Portfolio Overview

In [ ]:

new_df = parse_month(pd.read_csv(DATA / "new_customers_by_month.csv"))
act_df = parse_month(pd.read_csv(DATA / "active_customers_by_month.csv"))
vol_df = parse_month(pd.read_csv(DATA / "transactions_volume_by_month.csv"))
inst_df = parse_month(pd.read_csv(DATA / "installments_breakdown_by_month.csv"))
cat_df = parse_month(pd.read_csv(DATA / "merchant_categories_by_month.csv"))


In [ ]:

line(new_df, "month", "new_customers", "New customers by month", "nb_new_customers_by_month.png")
line(act_df, "month", "active_customers", "Active customers by month", "nb_active_customers_by_month.png")
bar(vol_df, "month", "tx_count", "Transactions count by month", "nb_tx_count_by_month.png")
line(vol_df, "month", "tx_amount", "Transactions amount by month", "nb_tx_amount_by_month.png")


## 2) Payment Analysis

In [ ]:

by_age = pd.read_csv(DATA / "dpd90_by_age_band.csv")
by_income = pd.read_csv(DATA / "dpd90_by_income_band.csv")
by_tx_month = pd.read_csv(DATA / "dpd90_by_tx_month.csv")
by_tx_month["tx_month"] = pd.to_datetime(by_tx_month["tx_month"], format="%Y-%m", errors="coerce")


In [ ]:

bar(by_age, "age_band", "dpd90_rate_pct", "DPD90 rate by age band", "nb_dpd90_by_age_band.png")
bar(by_income, "income_band", "dpd90_rate_pct", "DPD90 rate by income band", "nb_dpd90_by_income_band.png")
line(by_tx_month, "tx_month", "dpd90_rate_pct", "DPD90 rate by transaction month", "nb_dpd90_by_tx_month.png")


## 3) Vintage Analysis

In [ ]:

vint_cum = pd.read_csv(DATA / "vintage_curves_cumulative.csv")
# Draw up to 8 cohorts for readability
for coh in sorted(vint_cum["cohort_month"].unique())[:8]:
    sub = vint_cum[vint_cum["cohort_month"] == coh].sort_values("month_plus")
    line(sub, "month_plus", "dpd90_cum_pct", f"Vintage curve – cohort {coh}", f"nb_vintage_curve_{coh}.png")
